In [1]:
df = pd.read_csv("./data/total_data.csv")

In [2]:
df["시놉시스"]

0        크리스마스 시즌의 시카고. 말썽꾸러기 케빈(맥컬리 컬킨)은 자기의 치즈 피자를 먹은...
1        클라크 오스왈드(Clark Wilhelm Griswold, Jr.: 체비 체이스 분...
2        초심리를 연구하는 뉴욕의 괴짜교수 피터 밴크맨(빌 머레이)와 레이몬드 스탠드(댄 애...
3        고스트 버스터즈 4인방은 악마 '고저'로부터 세계를 구한 후 순탄한 생활을 하지 못...
4        조지 베일리는 베드폴드 마을 사람들을 위해 평생을 봉사하며 살아온 인물. 그는 항상...
                               ...                        
19139                    악을 징벌하는 대영웅으로 자라나는 특출한 소년 목진의 성장기
19140    영화에서는 잘만 풀리는 로맨스, 현실에선 왜 이리 어려울까? 도대체 ‘사랑이 뭔데!...
19141                            일본의 디바 탄생에 숨겨진 사랑과 이별 이야기
19142    여름, 미국 태평양 연안 북서부의 바다가 금광으로 변신한다! 값비싼 참치 떼를 잡기...
19143                         수수께끼의 재해인 세계 동시 도시 소멸로부터 5년.
Name: 시놉시스, Length: 19144, dtype: object

In [3]:
# 형태소 분리
import konlpy
from konlpy.tag import Okt
okt = Okt()

import re
def sub_special(s):
    return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',s)

STOP_WORDS = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
def morph_and_stopword(s):
    token_ls = []
    #형태소 분석
    tmp = okt.morphs(s, stem=True)

    #불용어 처리
    for token in tmp:
        if token not in STOP_WORDS:
            token_ls.append(token)
    return token_ls

df["시놉시스"] = df["시놉시스"].apply(sub_special).apply(morph_and_stopword)
df["시놉시스"] = df["시놉시스"].apply(lambda x: " ".join(x))

In [4]:
df["시놉시스"]

0        크리스마스 시즌 시카고 말썽 꾸러기 케빈 맥 컬리 컬킨 자기 치즈 피자 먹다 형 싸...
1        클라크 오스왈드 Clark Wilhelm Griswold Jr 체비 체이스 불다 식...
2        초 심리 연구 뉴욕 괴짜 교수 피터 배다 크다 맨 빌다 머레이 레이 몬드 스탠드 대...
3        고스트 버스터 즈 4 인방 악마 고 저 로부터 세계 구한 후 순탄 생활 을 못 하고...
4        조지 베 일리 베드 폴드 마을 사람 을 위해 평생 을 봉사 살아오다 인물 그 항상 ...
                               ...                        
19139                    악 을 징벌 대 영웅 자라나다 특출하다 소년 목 진의 성장기
19140    영화 에서는 잘만 풀리다 로맨스 현실 에선 왜 이리 어리다 도대체 사랑 뭔 데 라고...
19141                               일본 디바 탄생 숨기다 사랑 이별 이야기
19142    여름 미국 태평양 연안 북서 부의 바다 금광 변신 값 비싸다 참치 떼 잡기 위해 배...
19143                         수수께끼 재해 인 세계 동시 도시 소멸 로부터 5년
Name: 시놉시스, Length: 19144, dtype: object

In [5]:

df["장르"][df["장르"].isna()] = "['']"  # 이후에는 제거
df["장르"] = df["장르"].apply(lambda x: eval(x))
df["장르"] = df["장르"].apply(lambda x : (' ').join(x))

df["평점"] = df["평점"].astype("float")
df["평가수"] = df["평가수"].astype("int")

C = df["평점"].mean()
m = df["평가수"].quantile(0.6)
def weighted_vote_average(record):
    v = record["평가수"]
    R = record["평점"]
    
    return ((v/(v+m)) * R) + ((m/(m+v)) * C)
df["가중치평점"] = df.apply(weighted_vote_average, axis=1)

from ast import literal_eval
def literal(x):
    return literal_eval(x)

def sum_list(x):
    ls = []
    for i in x[:10]:
        text = i.strip()
        text = text.replace('.', '')
        text = text.replace("'", '_')
        text = text.replace(" ", '_')
        ls.append(text)
    return " ".join(ls)

df["감독"][df["감독"].isna()] = "['']"
df["감독"] = df["감독"].apply(literal)
df["감독"] = df["감독"].apply(sum_list)

df["출연진"][df["출연진"].isna()] = "['']"
df["출연진"] = df["출연진"].apply(lambda x: eval(x))
df["출연진"] = df["출연진"].apply(sum_list)

df["장르_종류"] = df["장르"] + " " + df["종류"]
df["특징"] = df["장르"] + " " + df["종류"] + " " + df["감독"] + " " + df["출연진"] + " " + df["시놉시스"]

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer()

# 예능

In [7]:
# df_1 = 예능 인지
# df_2 = 예능 아님

In [10]:
# 장르
tfidf_matrix = tfidf.fit_transform(df.장르_종류)
genre_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
temp = pd.DataFrame(genre_sim_sorted_ind[:, :21])
temp.to_csv("./data/장르_matrix.csv", index=False)

In [11]:
# 감독
tfidf_matrix = tfidf.fit_transform(df.감독)
genre_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
temp = pd.DataFrame(genre_sim_sorted_ind[:, :21])
temp.to_csv("./data/감독_matrix.csv", index=False)

In [12]:
# 출연진
tfidf_matrix = tfidf.fit_transform(df.출연진)
genre_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
temp = pd.DataFrame(genre_sim_sorted_ind[:, :21])
temp.to_csv("./data/출연진_matrix.csv", index=False)

In [61]:
tfidf = TfidfVectorizer(ngram_range=(1,2))
# 시놉시스
tfidf_matrix = tfidf.fit_transform(df.시놉시스)
genre_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
temp = pd.DataFrame(genre_sim_sorted_ind[:, :21])
temp.to_csv("./data/시놉시스_matrix.csv", index=False)

In [8]:
# 특징
tfidf = TfidfVectorizer(ngram_range=(1,2))
tfidf_matrix = tfidf.fit_transform(df.특징)
genre_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]

In [9]:
# genre_sim_sorted_ind_105 = genre_sim_sorted_ind[:, :105]
# genre_sim_sorted_ind_105 = genre_sim_sorted_ind_105.reshape((-1, 21))
# temp = pd.DataFrame(genre_sim_sorted_ind_105)
# temp.to_csv("./data/특징_matrix_100.csv", index=False)

# genre_sim_sorted_ind_84 = genre_sim_sorted_ind[:, :84]
# genre_sim_sorted_ind_84 = genre_sim_sorted_ind_84.reshape((-1, 21))
# temp = pd.DataFrame(genre_sim_sorted_ind_84)
# temp.to_csv("./data/특징_matrix_80.csv", index=False)

genre_sim_sorted_ind_63 = genre_sim_sorted_ind[:, :61]
# genre_sim_sorted_ind_63 = genre_sim_sorted_ind_63.reshape((-1, 21))
temp = pd.DataFrame(genre_sim_sorted_ind_63)
temp.to_csv("./data/특징_matrix_60.csv", index=False)

# genre_sim_sorted_ind_42 = genre_sim_sorted_ind[:, :42]
# genre_sim_sorted_ind_42 = genre_sim_sorted_ind_42.reshape((-1, 21))
# temp = pd.DataFrame(genre_sim_sorted_ind_42)
# temp.to_csv("./data/특징_matrix_40.csv", index=False)

# genre_sim_sorted_ind_21 = genre_sim_sorted_ind[:, :21]
# temp = pd.DataFrame(genre_sim_sorted_ind_21)
# temp.to_csv("./data/특징_matrix_20.csv", index=False)

In [8]:

temp = pd.DataFrame(genre_sim_sorted_ind[:, :61])
temp.to_csv("./data/특징_matrix.csv", index=False)

# 추천 시스템

832      13일의 금요일 6: 제이슨 살아있다
3407               박물관이 살아있다!
3579              박물관이 살아있다 2
6808        박물관이 살아있다: 비밀의 무덤
6980                장난감이 살아있다
10714            박물관이 진짜 살아있다
11248                   #살아있다
16870                언니는 살아있다
Name: 제목, dtype: object

In [231]:

titles = ["스파이더맨", "스파이더맨 2"]

In [13]:
import pandas as pd
import numpy as np

def recommendations(titles, genre = "특징"):
    
    # titles 예능인지 아닌지
    
    indexes = []
    for title in titles:
        indexes.append(df[df.제목 == title].index[0])
    
    similar_indexes = pd.read_csv(f"./data/{genre}_matrix.csv")
    similar_indexes = np.array(similar_indexes)[indexes,:]
    similar_indexes = similar_indexes.reshape(-1)
    similar_indexes = set(similar_indexes)  # 중복된 영화들을 제거/ 가중치 부여 # ["마블", "마블", "대부", "배트맨"]
    similar_indexes = np.array(list(similar_indexes))
    for i in indexes:
        similar_indexes = similar_indexes[similar_indexes != i]  # 타이틀 기준 제거
    print(df[genre][indexes[0]])
    return df.iloc[similar_indexes].sort_values("가중치평점", ascending=False)[:20]
    # return df.iloc[similar_indexes][:20]

In [235]:
recommendations(titles)[["제목"]][:10]

[11779  5639  9739 14229  5014  8856 10520  2584 11039 11042  1963 14772
  3382  6465 12102  6856  3280  4946 16853  3287  3929  6235  4831  5606
  2920  4973   366  3567  1657  1659 12285  2431] 1657
[11779  5639  9739 14229  5014  8856 10520  2584 11039 11042  1963 14772
  3382  6465 12102  6856  3280  4946 16853  3287  3929  6235  4831  5606
  2920  4973   366  3567  1659 12285  2431] 1659
판타지 로맨스 액션 드라마 SF 영화 Sam_Raimi Tobey_Maguire Willem_Dafoe Kirsten_Dunst James_Franco Rosemary_Harris JK_Simmons Cliff_Robertson Gerry_Becker Bill_Nunn Jack_Betts 평범하다 내 성적 인 고등학생 피터 파커 우연히 방사 능 감염 되다 거미 물리다 다그 후 피터 손 에서 거미줄 튀어 나오다 벽 을 기어 오르다 수 있다 거미 같다 능력 을 갖다 되다 다가오다 위험 을 본능 적 감지 초감각 엄청나다 파워 소유 되다 것 이다 피터 짝사랑 메리 제인 관심 을 끄다 위해 멋지다 스포츠카 구입 초능력 을 처음 사용 다 그렇다 사랑 베다 아저씨 죽음 을 계기 로 그 힘 을 악의 세력 대항 쓰기 로 결심 한편 피터 절친 해리 오스본 아버지 인 노 만은 실험 도중 가스 중독 되어다 악의 화신 그린 고블린 변하다


,제목
11039,스파이더맨: 뉴 유니버스
366,어느 날 밤에 생긴 일
9739,스파이더맨: 홈커밍
11042,스파이더맨: 파 프롬 홈
2920,러브 오브 시베리아
5014,제인 에어
4946,크로니클
4831,어메이징 스파이더맨
1963,피구의 제왕
8856,피터와 드래곤


In [ ]:
df

In [219]:
df[df.제목 == "스파이더맨"]

,제목,원제,개봉일,스트리밍,대여,구매,평점,평가수,장르,종류,재생 시간,감독,출연진,시놉시스,이미지,가중치평점,장르_종류,특징
1657,스파이더맨,Spider-Man,2002,"{'Netflix': ['정액제', 'HD'], 'wavve': ['정액제', 'H...","{'wavve': ['₩1,300', 'HD'], 'Naver Store': ['₩...","{'Naver Store': ['₩5,000', 'HD'], 'wavve': ['₩...",7.3,749,판타지 로맨스 액션 드라마 SF,영화,2시간 1분,Sam_Raimi,Tobey_Maguire Willem_Dafoe Kirsten_Dunst James...,평범하다 내 성적 인 고등학생 피터 파커 우연히 방사 능 감염 되다 거미 물리다 다...,https://images.justwatch.com/poster/184978760/...,7.292246,판타지 로맨스 액션 드라마 SF 영화,판타지 로맨스 액션 드라마 SF 영화 Sam_Raimi Tobey_Maguire W...
15353,스파이더맨,Spider-Man,1981,"{'Disney Plus': ['1개', 'HD']}",NaN,NaN,7.0,1,판타지 SF 애니메이션 액션 가족,tv프로그램,22분,,Ted_Schwartz Morgan_Lofting Linda_Gary Mona_Ma...,SpiderMan is an American animated TV series ba...,https://images.justwatch.com/poster/242839959/...,6.072202,판타지 SF 애니메이션 액션 가족 tv프로그램,판타지 SF 애니메이션 액션 가족 tv프로그램 Ted_Schwartz Morgan...
16853,스파이더맨,Marvel's Spider-Man,2017,"{'Disney Plus': ['2개', 'HD']}",NaN,NaN,6.1,2,SF 가족 애니메이션 액션,tv프로그램,24분,,Robbie_Daymond Nadji_Jeter Max_Mittleman Fred_...,피터 파커 절친 친구 인 해리 오스본 을 포함 모든 사람 로부터 평범하다 학생 이자...,https://images.justwatch.com/poster/67863254/s...,5.926835,SF 가족 애니메이션 액션 tv프로그램,SF 가족 애니메이션 액션 tv프로그램 Robbie_Daymond Nadji_Je...


In [125]:
recommendations(["스파이더맨"], "감독")[["제목", "감독"]][:10]

Sam_Raimi


,제목,감독
254,이블 데드 2,Sam_Raimi
177,이블 데드,Sam_Raimi
2092,심플 플랜,Sam_Raimi
1659,스파이더맨 2,Sam_Raimi
2200,기프트,Sam_Raimi
3584,드래그 미 투 헬,Sam_Raimi
1995,퀵 앤 데드,Sam_Raimi
6384,찰리 컨트리맨,Fredrik_Bond
6692,오즈: 그레이트 앤드 파워풀,Sam_Raimi
3287,스파이더맨 3,Sam_Raimi


In [126]:
recommendations(["스파이더맨"], "출연진")[["제목", "출연진"]][:10]

Tobey_Maguire Willem_Dafoe Kirsten_Dunst James_Franco Rosemary_Harris JK_Simmons Cliff_Robertson Gerry_Becker Bill_Nunn Jack_Betts


,제목,출연진
1858,라스베가스의 공포와 혐오,Johnny_Depp Benicio_del_Toro Tobey_Maguire Chr...
1659,스파이더맨 2,Tobey_Maguire Kirsten_Dunst James_Franco Alfre...
3524,악마가 너의 죽음을 알기 전에,Philip_Seymour_Hoffman Ethan_Hawke Albert_Finn...
6671,위대한 개츠비,Leonardo_DiCaprio Tobey_Maguire Carey_Mulligan...
3408,브라더스,Tobey_Maguire Jake_Gyllenhaal Natalie_Portman ...
668,브라질에서 온 소년,Gregory_Peck Laurence_Olivier James_Mason Lill...
15780,마블 얼티밋 스파이더맨,Drake_Bell Misty_Lee JK_Simmons Chi_McBride
6871,레이버 데이,Kate_Winslet Josh_Brolin Gattlin_Griffith Tobe...
1039,피크닉,William_Holden Kim_Novak Rosalind_Russell Bett...
2200,기프트,Cate_Blanchett Giovanni_Ribisi Keanu_Reeves Ka...


In [127]:
recommendations(["스파이더맨"], "장르")[["제목", "장르"]][:10]

판타지 로맨스 액션 드라마 SF


,제목,장르
15758,왕좌의 게임,드라마 액션 SF 판타지 로맨스
11110,Zack Snyder's Justice League,액션 판타지 SF 드라마
1659,스파이더맨 2,판타지 액션 드라마 SF
11108,승리호,SF 드라마 판타지 액션
6263,감기,드라마 로맨스 판타지 액션 SF 스릴러
8710,배트맨 대 슈퍼맨: 저스티스의 시작,판타지 액션 드라마 SF
2315,페노메논,판타지 SF 드라마 로맨스
2945,가메라2: 레기온 내습,드라마 판타지 액션 SF
11511,음양사: 청아집,액션 드라마 판타지 SF
16791,청운지,액션 SF 드라마 판타지 로맨스


In [128]:
recommendations(["스파이더맨"], "시놉시스")[["제목", "시놉시스"]][:10]

평범하다 내 성적 인 고등학생 피터 파커 우연히 방사 능 감염 되다 거미 물리다 다그 후 피터 손 에서 거미줄 튀어 나오다 벽 을 기어 오르다 수 있다 거미 같다 능력 을 갖다 되다 다가오다 위험 을 본능 적 감지 초감각 엄청나다 파워 소유 되다 것 이다 피터 짝사랑 메리 제인 관심 을 끄다 위해 멋지다 스포츠카 구입 초능력 을 처음 사용 다 그렇다 사랑 베다 아저씨 죽음 을 계기 로 그 힘 을 악의 세력 대항 쓰기 로 결심 한편 피터 절친 해리 오스본 아버지 인 노 만은 실험 도중 가스 중독 되어다 악의 화신 그린 고블린 변하다


,제목,시놉시스
11039,스파이더맨: 뉴 유니버스,평범하다 10 대다 마일 스 모랄레스 우연히 방사 능 거미 물리다 스파이더맨 능력 ...
366,어느 날 밤에 생긴 일,재산 을 노리 고 구혼 을 비행사 인 킹 웨슬리 결혼 엘리 아버지 알렉산더 반대 하...
9739,스파이더맨: 홈커밍,토니 스타크 제안 을 받다 시빌 워 참여 피터 파커 다음 임무 언 제 일지 설레다 ...
11042,스파이더맨: 파 프롬 홈,어벤져스 멤버 타 노스 와의 대 접전 을 마치다 뒤 지난 5년 동안 사라지다 사람 ...
1659,스파이더맨 2,우연 사고 로 특별하다 능력 을 갖다 되다 피터 파커 대학생 슈퍼 히어로 신분 을 ...
4831,어메이징 스파이더맨,어리다 적 사라지다 부모 대신 삼촌 내외 살 고 있다 피터 파커 평범하다 학교 생활...
8856,피터와 드래곤,갑작스럽다 사고 로 부모 잃다 비밀 로 가득하다 숲 속 에서 혼자 살아가다 되다 꼬...
11779,고공 침투: 노 리턴,항공기 부품 을 설치 기업 기술자 인 피터 어느 날 상사 제이슨 으로부터 부품 교체...
5087,커플로 살아남기,홀리 캐서린 헤이글 에릭 조 쉬 듀하멜 어느 날 서로 결혼 그 친구 인 피터 헤이스...
3287,스파이더맨 3,대중 사랑 을 듬뿍 받다 진정하다 영웅 거듭 난 스파이더맨 피터 파커 어느 날 외계...


In [66]:
recom = recommendations(["스파이더맨"], "시놉시스")[["제목", "시놉시스"]]

평범하다 내 성적 인 고등학생 피터 파커 우연히 방사 능 감염 되다 거미 물리다 다그 후 피터 손 에서 거미줄 튀어 나오다 벽 을 기어 오르다 수 있다 거미 같다 능력 을 갖다 되다 다가오다 위험 을 본능 적 감지 초감각 엄청나다 파워 소유 되다 것 이다 피터 짝사랑 메리 제인 관심 을 끄다 위해 멋지다 스포츠카 구입 초능력 을 처음 사용 다 그렇다 사랑 베다 아저씨 죽음 을 계기 로 그 힘 을 악의 세력 대항 쓰기 로 결심 한편 피터 절친 해리 오스본 아버지 인 노 만은 실험 도중 가스 중독 되어다 악의 화신 그린 고블린 변하다


평범하다 내 성적 인 고등학생 피터 파커 우연히 방사 능 감염 되다 거미 물리다 다그 후 피터 손 에서 거미줄 튀어 나오다 벽 을 기어 오르다 수 있다 거미 같다 능력 을 갖다 되다 다가오다 위험 을 본능 적 감지 초감각 엄청나다 파워 소유 되다 것 이다 피터 짝사랑 메리 제인 관심 을 끄다 위해 멋지다 스포츠카 구입 초능력 을 처음 사용 다 그렇다 사랑 베다 아저씨 죽음 을 계기 로 그 힘 을 악의 세력 대항 쓰기 로 결심 한편 피터 절친 해리 오스본 아버지 인 노 만은 실험 도중 가스 중독 되어다 악의 화신 그린 고블린 변하다

In [69]:
recom["시놉시스"][366]

'재산 을 노리 고 구혼 을 비행사 인 킹 웨슬리 결혼 엘리 아버지 알렉산더 반대 하고 요트 가두다 두다 엘리 헤엄 을 치다 해안 도망치다 나오다 뉴욕 가다 그레이 하운드 버스 오르다 그 버스 에는 근무 중 술 을 마시다 해고 당 신문 기자 피터 맨 뒷 좌석 자리 잡고 있다 엘리 피터 맨 뒤 좌석 나란하다 앉다 되다 여행 도중 엘리 소지품 들다 가방 을 잃어버리다 음에도 신고 거부 피터 엘리 신분 대해 의심 을 되다 그레 하운드 로 계속 되다 여행 다음 날 둘 잠시 쉰 휴게실 에서 느긋하다 아침 식사 엘리 피터 버스 놓치다 되다 그렇다 와중 피터 엘리 신분 을 알 게 되다 엘리 피터 에게 뉴욕 도착 도와주다 자신 대한 특종 기사 쓰다 해 주다 거래 제안 하 지만 부잣집 딸 이나 부르다 타다 결혼 킹 웨슬리 같다 사람 한테 전혀 관심 없다 퉁 명 스럽게 제안 을 거절 하지만 북쪽 향 여행 에서 몇 가지 우연찮 사고 일어나다 되다 피터 엘리 여러 곤경 같이 빠지다 되다 그렇다 둘 서로 사랑 빠지다 되다'

In [207]:
recommendations(["스파이더맨"])[["제목", "특징"]]

판타지 로맨스 액션 드라마 SF 영화 Sam_Raimi Tobey_Maguire Willem_Dafoe Kirsten_Dunst James_Franco Rosemary_Harris JK_Simmons Cliff_Robertson Gerry_Becker Bill_Nunn Jack_Betts 평범하다 내 성적 인 고등학생 피터 파커 우연히 방사 능 감염 되다 거미 물리다 다그 후 피터 손 에서 거미줄 튀어 나오다 벽 을 기어 오르다 수 있다 거미 같다 능력 을 갖다 되다 다가오다 위험 을 본능 적 감지 초감각 엄청나다 파워 소유 되다 것 이다 피터 짝사랑 메리 제인 관심 을 끄다 위해 멋지다 스포츠카 구입 초능력 을 처음 사용 다 그렇다 사랑 베다 아저씨 죽음 을 계기 로 그 힘 을 악의 세력 대항 쓰기 로 결심 한편 피터 절친 해리 오스본 아버지 인 노 만은 실험 도중 가스 중독 되어다 악의 화신 그린 고블린 변하다


,제목,특징
11779,고공 침투: 노 리턴,액션 드라마 스릴러 MadeinEurope 영화 Kevin_Glynn Bernard...
9739,스파이더맨: 홈커밍,SF 액션 영화 Jon_Watts Tom_Holland Michael_Keaton ...
14229,싸이코패스 스토커 살인,공포 영화 Rocky_Costanzo Quinton_Jackson Steve_De_...
8856,피터와 드래곤,판타지 코미디 드라마 가족 액션 영화 David_Lowery Oakes_Fegley...
11039,스파이더맨: 뉴 유니버스,애니메이션 SF 액션 코미디 가족 판타지 영화 Rodney_Rothman Peter...
11042,스파이더맨: 파 프롬 홈,SF 액션 코미디 영화 Jon_Watts Tom_Holland Samuel_L_Ja...
1963,피구의 제왕,코미디 스포츠 Made in Europe 영화 Rawson_Marshall_Thur...
14772,타임스톤: 시간여행자,스릴러 드라마 영화 Lorena_Villarreal Rupert_Graves Joh...
6465,메가 스파이더,SF 스릴러 액션 코미디 영화 Mike_Mendez Greg_Grunberg Lom...
12102,싱글 올 더 웨이,로맨스 코미디 드라마 영화 Michael_Mayer Michael_Urie Phil...


In [130]:
# recom_1 = recommendations(["스파이더맨", "What If...?"], "장르")
# recom_2 = recommendations(["스파이더맨", "What If...?"], "감독")
# recom_3 = recommendations(["스파이더맨", "What If...?"], "출연진")
# recom_4 = recommendations(["스파이더맨", "What If...?"], "시놉시스")
# recom_5 = recommendations(["스파이더맨", "What If...?"], "특징")
recom_1 = recommendations(["스파이더맨"], "장르")
recom_2 = recommendations(["스파이더맨"], "감독")
recom_3 = recommendations(["스파이더맨"], "출연진")
recom_4 = recommendations(["스파이더맨"], "시놉시스")
recom_5 = recommendations(["스파이더맨"], "특징")

판타지 로맨스 액션 드라마 SF
Sam_Raimi
Tobey_Maguire Willem_Dafoe Kirsten_Dunst James_Franco Rosemary_Harris JK_Simmons Cliff_Robertson Gerry_Becker Bill_Nunn Jack_Betts
평범하다 내 성적 인 고등학생 피터 파커 우연히 방사 능 감염 되다 거미 물리다 다그 후 피터 손 에서 거미줄 튀어 나오다 벽 을 기어 오르다 수 있다 거미 같다 능력 을 갖다 되다 다가오다 위험 을 본능 적 감지 초감각 엄청나다 파워 소유 되다 것 이다 피터 짝사랑 메리 제인 관심 을 끄다 위해 멋지다 스포츠카 구입 초능력 을 처음 사용 다 그렇다 사랑 베다 아저씨 죽음 을 계기 로 그 힘 을 악의 세력 대항 쓰기 로 결심 한편 피터 절친 해리 오스본 아버지 인 노 만은 실험 도중 가스 중독 되어다 악의 화신 그린 고블린 변하다
판타지 로맨스 액션 드라마 SF 영화 Sam_Raimi Tobey_Maguire Willem_Dafoe Kirsten_Dunst James_Franco Rosemary_Harris JK_Simmons Cliff_Robertson Gerry_Becker Bill_Nunn Jack_Betts 평범하다 내 성적 인 고등학생 피터 파커 우연히 방사 능 감염 되다 거미 물리다 다그 후 피터 손 에서 거미줄 튀어 나오다 벽 을 기어 오르다 수 있다 거미 같다 능력 을 갖다 되다 다가오다 위험 을 본능 적 감지 초감각 엄청나다 파워 소유 되다 것 이다 피터 짝사랑 메리 제인 관심 을 끄다 위해 멋지다 스포츠카 구입 초능력 을 처음 사용 다 그렇다 사랑 베다 아저씨 죽음 을 계기 로 그 힘 을 악의 세력 대항 쓰기 로 결심 한편 피터 절친 해리 오스본 아버지 인 노 만은 실험 도중 가스 중독 되어다 악의 화신 그린 고블린 변하다


In [131]:

# recom = pd.concat([recom_1, recom_2, recom_3, recom_4], ignore_index=True)
recom = pd.concat([recom_1, recom_2, recom_3, recom_4, recom_5], ignore_index=True)

In [132]:
recom["count"] = 1

In [133]:
recom.groupby(["제목"]).sum().sort_values(["count", "가중치평점"], ascending=False)[:20][["count"]]

,count
제목,
스파이더맨 2,5
스파이더맨 3,4
스파이더맨: 뉴 유니버스,2
어느 날 밤에 생긴 일,2
스파이더맨: 홈커밍,2
스파이더맨: 파 프롬 홈,2
어메이징 스파이더맨,2
기프트,2
피터와 드래곤,2


In [139]:
recom_5["제목"]

11039      스파이더맨: 뉴 유니버스
366         어느 날 밤에 생긴 일
9739          스파이더맨: 홈커밍
11042      스파이더맨: 파 프롬 홈
1659             스파이더맨 2
4831          어메이징 스파이더맨
1963              피구의 제왕
8856             피터와 드래곤
11779        고공 침투: 노 리턴
3287             스파이더맨 3
6856              윈터스 테일
12102          싱글 올 더 웨이
16853              스파이더맨
3280                도협대전
14772        타임스톤: 시간여행자
14229       싸이코패스 스토커 살인
2431         브링 다운 더 하우스
5606              노바디 웍스
6465             메가 스파이더
12285    크리미널 게임: 보석 사기단
Name: 제목, dtype: object

In [135]:
df[["제목", "장르", "감독", "출연진", "시놉시스"]]

,제목,장르,감독,출연진,시놉시스
0,나 홀로 집에,코미디 가족,Chris_Columbus,Macaulay_Culkin Joe_Pesci Daniel_Stern John_He...,크리스마스 시즌 시카고 말썽 꾸러기 케빈 맥 컬리 컬킨 자기 치즈 피자 먹다 형 싸...
1,크리스마스 대소동,코미디 Made in Europe,Jeremiah_S_Chechik,Chevy_Chase Beverly_D_Angelo Juliette_Lewis Jo...,클라크 오스왈드 Clark Wilhelm Griswold Jr 체비 체이스 불다 식...
2,고스트 버스터즈,판타지 SF 액션 코미디,Ivan_Reitman,Bill_Murray Dan_Aykroyd Sigourney_Weaver Harol...,초 심리 연구 뉴욕 괴짜 교수 피터 배다 크다 맨 빌다 머레이 레이 몬드 스탠드 대...
3,고스트 버스터즈 2,판타지 액션 코미디 SF,Ivan_Reitman,Bill_Murray Dan_Aykroyd Sigourney_Weaver Harol...,고스트 버스터 즈 4 인방 악마 고 저 로부터 세계 구한 후 순탄 생활 을 못 하고...
4,멋진 인생,판타지 드라마 가족,Frank_Capra,James_Stewart Donna_Reed Lionel_Barrymore Thom...,조지 베 일리 베드 폴드 마을 사람 을 위해 평생 을 봉사 살아오다 인물 그 항상 ...
...,...,...,...,...,...
19139,대주재,SF,Zhang_Meng Cheng_Feng,Roy_Wang Ouyang_Nana Luo_Mingjie,악 을 징벌 대 영웅 자라나다 특출하다 소년 목 진의 성장기
19140,카란 조하르의 사랑이 뭔데!,RealityTV,,Karan_Johar,영화 에서는 잘만 풀리다 로맨스 현실 에선 왜 이리 어리다 도대체 사랑 뭔 데 라고...
19141,M 사랑해야 할 사람이 있어서,RealityTV 음악,Takao_Kinoshita Manabu_Asô,Kalen_Anzai Shohei_Miura Katsunori_Takahashi M...,일본 디바 탄생 숨기다 사랑 이별 이야기
19142,배틀피시,RealityTV,,,여름 미국 태평양 연안 북서 부의 바다 금광 변신 값 비싸다 참치 떼 잡기 위해 배...


In [104]:
recom_5[["제목", "장르", "가중치평점"]]

,제목,장르,가중치평점
17216,주술회전,SF 판타지 스릴러 애니메이션 액션,8.406693
11039,스파이더맨: 뉴 유니버스,애니메이션 SF 액션 코미디 가족 판타지,8.377196
17220,로키,SF 판타지 액션,8.259841
17268,할리 퀸,SF 코미디 범죄 판타지 애니메이션 액션,8.056709
366,어느 날 밤에 생긴 일,코미디 로맨스,8.012243
16419,셰프의 테이블,다큐멘터리,7.908945
15581,은혼,액션 SF 애니메이션 코미디,7.680084
16052,에이전트 오브 쉴드,액션 SF 드라마 스릴러,7.468831
9739,스파이더맨: 홈커밍,SF 액션,7.388819
11042,스파이더맨: 파 프롬 홈,SF 액션 코미디,7.384557


In [215]:
recommendations(["좋은 놈, 나쁜 놈, 이상한 놈"], "특징")[["제목", "시놉시스"]]

서부 코미디 전쟁 스릴러 액션 영화 Kim_Jee-woon Song_Kang-ho Lee_Byung-hun Jung_Woo-sung Yoon_Je-moon Song_Young-chang Lee_Chung-ah Kim_Kwang-il Ma_Dong-seok Jo_Kyeong-hun Lee_Hang-soo 1930년 대 다양하다 인종 뒤 엉 키 고 총 칼 난무 무법천지 만주 에서 각자 다른 방식 격동 기르다 살아가다 조선 풍운아 세 명의 남자 운명 처럼 맞닥뜨 린다 돈 되다 건 뭐 든 사냥 현상금 사냥꾼 박도 원 최고 아니다 차다 수 없다 마적 단 두목 박창 잡초 같다 생명력 독고 다 열차 털이범 윤 태구 서로 정체 모르다 채 태구 열차 털다 발견 지도 차지 위해 대륙 을 누비 추 격전 을 펼치다 정체 불명 지도 장 을 두르다 싸다 엇갈리다 추측 속 일본군 마적 단 까지 레이스 가담 되다


,제목,시놉시스
4848,악마를 보았다,국정원 경호 요원 수현 약혼녀 주연 잔인하다 살해 당하다 자신 에게 가장 소중하다 ...
8700,부산행,별거 중인 펀드 매니저 석우 소원 딸 수안 생일 소원 을 들어주다 위해 아내 있다 ...
3687,달콤한 인생,서울 하늘 켜다 섬 처럼 뜨다 있다 호텔 스카이 라운지 그 곳 냉철하다 명민하다 완...
4995,슈퍼 에이트,1979년 기차 플랫폼 부근 에서 슈퍼 8 mm 카메라 로 영화 촬영 6 명의 아이...
7778,슬로우 웨스트,19 세기 서부 개척 시 대의 현상금 사냥꾼 사일러 스마 크다 패스 베다 더 숲 에...
8850,밀정,1920년 대 일제강점기 조선인 출신 일본 경찰 이정 추다 무장 독립운동 단체 의열...
3727,트랜스 시베리아,중국 러시아 광활하다 대륙 을 횡단하 시베리아 횡단 열차 미국 에서 온 로이 제시 ...
2843,조용한 가족,서울 에서 그리다 멀다 않다 한적하다 곳 에서 장사 라고는 번 해보다 적 없다 일 ...
5582,검우강호,명나라 시대 8 백년 전 사라지다 달마 유해 차지 위해 전국 검객 곳 모여들다 황실...
11096,낙원의 밤,범죄 조직 에이스 태구 사랑 누나 조카 의문 교통사고 로 한순간 잃다 참혹하다 비극...


In [176]:
recom_5 = recommendations(["우리 결혼했어요"], "장르")

RealityTV


In [188]:
df.shape

(19144, 18)

In [ ]:
if 타이틀에 'RealityTV' 있으면:
    # df[df.장르.str.contains('RealityTV')]
else:
    # df[~df.장르.str.contains('RealityTV')]

In [191]:
df["종류"].value_counts()

영화        15351
tv프로그램     3793
Name: 종류, dtype: int64

In [189]:
df[df.장르.str.contains('RealityTV')]

,제목,원제,개봉일,스트리밍,대여,구매,평점,평가수,장르,종류,재생 시간,감독,출연진,시놉시스,이미지,가중치평점,장르_종류,특징
7914,테라스 하우스 : 클로징 도어,テラスハウス クロージング・ドア,2015,"{'Watcha': ['정액제', 'HD']}","{'wavve': ['₩1,540', 'HD'], 'Naver Store': ['₩...","{'wavve': ['₩6,050', 'HD'], 'Naver Store': ['₩...",5.7,0,로맨스 RealityTV 드라마,영화,2시간 9분,Masato_Maeda,You Reina_Triendl Azusa_Babazono Ryota_Yamasat...,청춘 남녀 테라스 하우스 에서 생활 짝 을 찾아가다 내용 쇼 보여주다 영화,https://images.justwatch.com/poster/140534589/...,5.840253,로맨스 RealityTV 드라마 영화,로맨스 RealityTV 드라마 영화 Masato_Maeda You Reina_Tr...
8073,4K Fireplace,NaN,2015,"{'Netflix': ['정액제', '4K']}",NaN,NaN,7.7,0,가족 RealityTV,영화,3시간 1분,,fireplace,Enjoy this crackling fireplace during a holida...,https://images.justwatch.com/poster/69766552/s...,5.840253,가족 RealityTV 영화,가족 RealityTV 영화 fireplace Enjoy this cracklin...
11566,디센던츠 3,Descendants 3,2019,"{'Disney Plus': ['정액제', 'HD']}",NaN,NaN,6.6,7,판타지 드라마 가족 스릴러 액션 RealityTV,영화,1시간 46분,Kenny_Ortega,Dove_Cameron Cameron_Boyce Sofia_Carson Booboo...,선 악의 대립 악당 아이 인 말 이비 카를로스 제이 잃어버리다 제도 오 라돈 신입생...,https://images.justwatch.com/poster/157544645/...,6.323728,판타지 드라마 가족 스릴러 액션 RealityTV 영화,판타지 드라마 가족 스릴러 액션 RealityTV 영화 Kenny_Ortega Do...
11803,당신과 자연의 대결: 사파리 대탈주,Animals on the Loose: A You vs. Wild Interacti...,2021,"{'Netflix': ['정액제', 'HD']}",NaN,NaN,5.8,0,액션 RealityTV,영화,45분,Ben_Simms,Bear_Grylls,야생 동물 보호 구역 에서 동물 탈출 사라지다 동물 을 찾다 안전하다 곳 보내다 위...,https://images.justwatch.com/poster/241341452/...,5.840253,액션 RealityTV 영화,액션 RealityTV 영화 Ben_Simms Bear_Grylls 야생 동물 보호...
12269,배드 트립,Bad Trip,2020,"{'Netflix': ['정액제', 'HD']}",NaN,NaN,6.6,21,코미디 RealityTV,영화,1시간 27분,Kitao_Sakurai,Eric_André Lil_Rel_Howery Tiffany_Haddish Mich...,평범하다 여행 재미없다 뉴욕 가다 길 두 친구 못 되다 장난 을 꾸미다 몰래카메라 ...,https://images.justwatch.com/poster/174509871/...,6.478440,코미디 RealityTV 영화,코미디 RealityTV 영화 Kitao_Sakurai Eric_André Lil_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19134,오 마이 펫 스타!,Pet Stars,2021,"{'Netflix': ['1개', '4K']}",NaN,NaN,5.7,0,RealityTV 가족,tv프로그램,27분,,,동물 인 플루 언 서 되다 수 있다 소셜미디어 최고 스타 일 동물 매니지먼트 회사 ...,https://images.justwatch.com/poster/244735367/...,5.840253,RealityTV 가족 tv프로그램,RealityTV 가족 tv프로그램 동물 인 플루 언 서 되다 수 있다 소셜미디...
19138,디마커스 패밀리 룰,DeMarcus Family Rules,2020,"{'Netflix': ['1개', '4K']}",NaN,NaN,6.3,0,RealityTV,tv프로그램,29분,,Jane_Alderson Dylan_DeMarcus Paige_Dennis Jay_...,일상 시트콤 같다 패밀리 래스칼 플래츠 멤버 제이 디마커스 그 아내 앨리슨 디마커스...,https://images.justwatch.com/poster/198355791/...,5.840253,RealityTV tv프로그램,RealityTV tv프로그램 Jane_Alderson Dylan_DeMarcus...
19140,카란 조하르의 사랑이 뭔데!,What the Love! with Karan Johar,2020,"{'Netflix': ['1개', '4K']}",NaN,NaN,0.0,0,RealityTV,tv프로그램,47분,,Karan_Johar,영화 에서는 잘만 풀리다 로맨스 현실 에선 왜 이리 어리다 도대체 사랑 뭔 데 라고...,https://images.justwatch.com/poster/167000161/...,5.840253,RealityTV tv프로그램,RealityTV tv프로그램 Karan_Johar 영화 에서는 잘만 풀리다 로맨...
19141,M 사랑해야 할 사람이 있어서,M 愛すべき人がいて,2020,"{'wavve': ['1개', 'HD'], 'Watcha': ['1개', 'HD']}",NaN,NaN,0.0,0,RealityTV 음악,tv프로그램,40분,Takao_Kinoshita Manabu_Asô,Kalen_Anzai Shohei_Miura Katsunori_Takahashi M...,일본 디바 탄생 숨기다 사랑 이별 이야기,https://images.justwatch.com/poster/246457551/...,5.840253,RealityTV 음악 tv프로그램,RealityTV 음악 tv프로그램 Takao_Kinoshita Manabu_Asô...


In [187]:
len(df[df.장르.str.contains('RealityTV')])

250

In [179]:
df.장르.value_counts()

드라마                           1312
코미디                            645
드라마 로맨스                        474
다큐멘터리                          452
드라마 코미디                        351
                              ... 
스릴러 공포 드라마 다큐멘터리 판타지 범죄 SF       1
스릴러 애니메이션 액션 드라마 전쟁              1
코미디 스릴러 판타지                      1
SF 스릴러 판타지 로맨스                   1
RealityTV 가족                     1
Name: 장르, Length: 4741, dtype: int64

In [223]:
# RealityTV 개별적으로 추천해주는...
recom_5 = recommendations(["배트맨"], "특징")

판타지 액션 영화 Tim_Burton Michael_Keaton Jack_Nicholson Kim_Basinger Robert_Wuhl Pat_Hingle Billy_Dee_Williams Michael_Gough Jack_Palance Jerry_Hall Tracey_Walter 범죄 부패 탐욕 도시 고담 정체 불명 배트맨 나타나다 공포 위협 으로부터 시민 을 구한 다 모든 시민 배트맨 활약 상 을 보다 그 대한 궁금증 을 더하다 간다 고 담시 모두 배트맨 대해 궁금하다 있다 때 전쟁 터 에서 활약 경험 까지 있다 유능하다 사진기 비키 베일 킴 베이 싱어 동료 기자 함께 배트맨 을 취재 그러나 비키 앞 에는 매력 적 인 의문 백만장자 브루스 웨인 마이클 키 틀다 등장 매력 적 이고 의문 투성이 인 젊다 부호 점점 비키 가까워지다 배트맨 인하다 모습 변하다 버리다 악당 조커 배트맨 에게 복수 다짐


In [224]:
# recom_5["장르"][366]

In [225]:
recom_5.sort_values("가중치평점", ascending=False)[["제목", "감독", "출연진", "장르", "시놉시스", "가중치평점"]]

,제목,감독,출연진,장르,시놉시스,가중치평점
3355,다크 나이트,Christopher_Nolan,Christian_Bale Heath_Ledger Michael_Caine Gary...,스릴러 액션 범죄 드라마,범죄 부정부패 제거 고 담시 지키다 배트맨 그 짐 고든 형사 패기 넘치다 고 담시 ...,8.993693
4854,다크 나이트 라이즈,Christopher_Nolan,Christian_Bale Michael_Caine Gary_Oldman Anne_...,범죄 스릴러 액션 드라마,배트맨 조커 와의 대결 을 끝 세상 에서 모습 을 감추다 8년 후 하비 덴트 죽음 ...,8.389802
3359,배트맨 비긴즈,Christopher_Nolan,Christian_Bale Michael_Caine Liam_Neeson Katie...,드라마 액션 범죄,브루스 웨인 어리다 시절 부모님 길거리 에서 피살 되다 것 을 눈 앞 에서 지켜보다...,8.190599
11110,Zack Snyder's Justice League,Zack_Snyder,Ben_Affleck Henry_Cavill Gal_Gadot Jason_Momoa...,액션 판타지 SF 드라마,슈퍼맨 죽다 지구 어둠 그림자 드리우다 마더 박스 차지 위해 빌런 스테픈울프 파라 ...,8.074026
16063,고담,,Ben_McKenzie Donal_Logue David_Mazouz Sean_Per...,SF 스릴러 액션 범죄 드라마,젊다 시절 고든 경감 브루스 웨인 부모 살해 자르다 복수 위해 범죄 부패 막다 고담...,7.765005
6262,레고 무비,"Christopher_Miller_,_Phil_Lord",Chris_Pratt Will_Ferrell Elizabeth_Banks Will_...,판타지 애니메이션 액션 코미디 가족 SF Made in Europe,배트맨 슈퍼맨 원더우먼 인어공주 초록 닌자 1980 몇 년 우주인 미켈란젤로 미켈란...,7.678249
9830,레고 배트맨 무비,Chris_McKay,Will_Arnett Zach_Galifianakis Michael_Cera Ros...,SF 판타지 액션 가족 범죄 코미디 로맨스 애니메이션 MadeinEurope,모 난 성격 때문 외톨이 로 지내다 알프레드 집사 에게 늘 걱정 만 가득 안기다 배...,7.259168
8710,배트맨 대 슈퍼맨: 저스티스의 시작,Zack_Snyder,Ben_Affleck Henry_Cavill Jesse_Eisenberg Gal_G...,판타지 액션 드라마 SF,슈퍼맨 조드 장군 격렬하다 전투 이후 메트로폴리스 파괴 되어다 슈퍼맨 세계 최고 논...,6.396658
8784,배트맨: 더 킬링 조크,Sam_Liu,Kevin_Conroy Mark_Hamill Tara_Strong Ray_Wise ...,범죄 액션 애니메이션 스릴러 드라마,조커 자신 관계 고민 배트맨 언젠가 늘다 자신 죽다 조커 죽다 끝 을 볼 것 이라고...,6.360719
9761,저스티스 리그,Zack_Snyder,Ben_Affleck Henry_Cavill Amy_Adams Gal_Gadot E...,판타지 SF 액션,인류 수호 인 슈퍼맨 사라지다 틈 을 노리 고 마더 박스 차지 위해 빌런 스테픈울프...,6.097606


In [146]:
recom_5[["제목", "감독", "출연진", "장르", "시놉시스", "가중치평점"]]

,제목,감독,출연진,장르,시놉시스,가중치평점
11779,고공 침투: 노 리턴,Kevin_Glynn,Bernard_Deegan Jordan_Coombes Patrick_O_Donnel...,액션 드라마 스릴러 MadeinEurope,항공기 부품 을 설치 기업 기술자 인 피터 어느 날 상사 제이슨 으로부터 부품 교체...,6.520126
9739,스파이더맨: 홈커밍,Jon_Watts,Tom_Holland Michael_Keaton Robert_Downey_Jr Ma...,SF 액션,토니 스타크 제안 을 받다 시빌 워 참여 피터 파커 다음 임무 언 제 일지 설레다 ...,7.388819
14229,싸이코패스 스토커 살인,Rocky_Costanzo,Quinton_Jackson Steve_De_Forest Noel_Gugliemi ...,공포,미국 서부 사막 평화롭다 시골 마을 브라운 스 빌다 난데없이 여성 끔찍하다 살해 되...,5.840253
8856,피터와 드래곤,David_Lowery,Oakes_Fegley Bryce_Dallas_Howard Wes_Bentley K...,판타지 코미디 드라마 가족 액션,갑작스럽다 사고 로 부모 잃다 비밀 로 가득하다 숲 속 에서 혼자 살아가다 되다 꼬...,6.642684
11039,스파이더맨: 뉴 유니버스,Rodney_Rothman Peter_Ramsey Bob_Persichetti,Shameik_Moore Jake_Johnson Hailee_Steinfeld Ma...,애니메이션 SF 액션 코미디 가족 판타지,평범하다 10 대다 마일 스 모랄레스 우연히 방사 능 거미 물리다 스파이더맨 능력 ...,8.377196
11042,스파이더맨: 파 프롬 홈,Jon_Watts,Tom_Holland Samuel_L_Jackson Jake_Gyllenhaal M...,SF 액션 코미디,어벤져스 멤버 타 노스 와의 대 접전 을 마치다 뒤 지난 5년 동안 사라지다 사람 ...,7.384557
1963,피구의 제왕,Rawson_Marshall_Thurber,Vince_Vaughn Christine_Taylor Ben_Stiller Rip_...,코미디 스포츠 Made in Europe,낡다 초라하다 가지다 것 없다 회원 에게는 최고 휴식 처 같다 애 버리다 조 체육관...,6.685848
14772,타임스톤: 시간여행자,Lorena_Villarreal,Rupert_Graves John_Noble Melina_Matthews Tina_...,스릴러 드라마,1970년 미국 에서 발사 미사일 멕시코 침묵 지대 추락 미사일 잔해 치우다 위해 ...,5.840253
6465,메가 스파이더,Mike_Mendez,Greg_Grunberg Lombardo_Boyar Clare_Kramer Ray_...,SF 스릴러 액션 코미디,군사 연구소 에서 비밀리 진행 되다 외계인 거미 유전자 변형 실험 중 돌연변이 괴물...,5.496456
12102,싱글 올 더 웨이,Michael_Mayer,Michael_Urie Philemon_Chambers Luke_Macfarlane...,로맨스 코미디 드라마,크리스마스 혼자 부모님 집 가기 싫다 피터 절친 에게 남자친구 인 척 해달라다 부탁...,6.080084


In [88]:
p = "배트맨"
df[df.제목.apply(lambda x: p in x)]["제목"]

61                                 배트맨
2114                          배트맨 앤 로빈
2158                           배트맨 포에버
3359                           배트맨 비긴즈
7072     레고 배트맨: 더 무비 - DC 수퍼히어로즈 유나이트
8066                        배트맨 vs. 로빈
8710               배트맨 대 슈퍼맨: 저스티스의 시작
8784                      배트맨: 더 킬링 조크
8979                       배트맨: 배드 블러드
9830                         레고 배트맨 무비
11392                           닌자 배트맨
Name: 제목, dtype: object

In [89]:
recommendations(["배트맨"])[["제목", "개봉일", "장르"]]

판타지 액션 영화 Tim_Burton Michael_Keaton Jack_Nicholson Kim_Basinger Robert_Wuhl Pat_Hingle Billy_Dee_Williams Michael_Gough Jack_Palance Jerry_Hall Tracey_Walter 범죄와 부패, 탐욕의 도시 고담에 정체불명의 배트맨이 나타나 공포와 위협으로부터 시민들을 구한다. 모든 시민들은 배트맨의 활약상을 보면서 그에 대한 궁금증을 더해간다. 고담시의 모두가 배트맨에 대해 궁금해 하고 있을때 전쟁터에서 활약한 경험까지 있는 유능한 사진기자 비키 베일(킴 베이싱어)은 동료 기자와 함께 배트맨을 취재한다. 그러나 비키 앞에는 매력적인 의문의 백만장자 브루스 웨인(마이클 키튼)이 등장한다. 매력적이고 의문투성이인 이 젊은 부호는 점점 비키와 가까워지고 배트맨으로 인해 모습이 변해버린 악당 조커는 배트맨에게 복수를 다짐한다.


,제목,개봉일,장르
3355,다크 나이트,2008,스릴러 액션 범죄 드라마
4854,다크 나이트 라이즈,2012,범죄 스릴러 액션 드라마
3359,배트맨 비긴즈,2005,드라마 액션 범죄
11110,Zack Snyder's Justice League,2021,액션 판타지 SF 드라마
16063,고담,2014,SF 스릴러 액션 범죄 드라마
6262,레고 무비,2014,판타지 애니메이션 액션 코미디 가족 SF Made in Europe
9830,레고 배트맨 무비,2017,SF 판타지 액션 가족 범죄 코미디 로맨스 애니메이션 MadeinEurope
8710,배트맨 대 슈퍼맨: 저스티스의 시작,2016,판타지 액션 드라마 SF
8784,배트맨: 더 킬링 조크,2016,범죄 액션 애니메이션 스릴러 드라마
9761,저스티스 리그,2017,판타지 SF 액션


['코미디', '가족']
[


ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [46]:
df["장르"][df["장르"].isna()] = "['']"  # 이후에는 제거
df["장르"] = df["장르"].apply(lambda x: eval(x))
pd.DataFrame(df["장르"].sum())[0].unique()

In [51]:
pd.DataFrame(df["장르"].sum())[0].unique()

array(['코미디', '가족', 'Made in Europe', '판타지', 'SF', '액션', '드라마', '범죄',
       '스릴러', '로맨스', '역사', '다큐멘터리', '공포', '애니메이션', '전쟁', '음악', '서부',
       '스포츠', 'Reality TV', 'MadeinEurope', 'RealityTV'], dtype=object)

In [ ]:
[
    '코미디', '가족', 'Made in Europe',
    '판타지', 'SF', '액션', '드라마',
    '범죄','스릴러', '로맨스', '역사',
    '다큐멘터리', '공포', '애니메이션',
    '전쟁', '음악', '서부', '스포츠',
    'Reality TV', 'MadeinEurope', 'RealityTV'
]

In [52]:
pd.DataFrame(df["장르"].sum())[0].value_counts()

드라마               10644
코미디                6224
스릴러                5079
액션                 4934
로맨스                3962
범죄                 2830
판타지                2669
SF                 2509
공포                 2367
애니메이션              2149
가족                 1899
MadeinEurope       1512
다큐멘터리              1428
Made in Europe     1084
역사                 1045
전쟁                  848
음악                  671
스포츠                 496
RealityTV           250
서부                  225
Reality TV            5
Name: 0, dtype: int64

In [60]:
df[df.장르.str.contains('Reality TV')][["제목"]]

,제목
3677,거친 녀석들
3697,넥스트
3874,남주기 아까운 그녀
5842,논스톱 액시던트
6186,죽이러 갑니다


In [65]:
df.개봉일.unique()

array([1990, 1989, 1984, 1946, 1973, 1985, 1983, 1977, 1993, 1965, 1972,
       1992, 1954, 1968, 1971, 1986, 1961, 1991, 1987, 1988, 1980, 1981,
       1947, 1950, 1976, 1964, 1967, 1940, 1962, 1939, 1938, 1949, 1982,
       1969, 1974, 1942, 1975, 1960, 1944, 1957, 1941, 1943, 1956, 1978,
       1951, 1979, 1959, 1952, 1958, 1963, 1953, 1948, 1926, 1945, 1955,
       1966, 1970, 1937, 1934, 1936, 1922, 1931, 1920, 1935, 1921, 1925,
       1930, 1933, 1927, 1932, 1923, 1928, 1929, 1919, 1916, 1999, 2002,
       2003, 2004, 1994, 2001, 1995, 1997, 1998, 2000, 1996, 2005, 2007,
       2008, 2006, 2009, 2012, 2010, 2011, 2014, 2013, 2015, 2016, 2017,
       2021, 2018, 2020, 2019, 2022], dtype=int64)

In [85]:
def years(x):
    if x < 1980:
        return 1900
    # elif 1950 <= x < 1960:
    #     return 1950
    # elif 1960 <= x < 1970:
    #     return 1960
    # elif 1970 <= x < 1980:
    #     return 1970
    elif 1980 <= x < 1990:
        return 1980
    elif 1990 <= x < 2000:
        return 1990
    elif 2000 <= x < 2010:
        return 2000
    elif 2010 <= x < 2020:
        return 2010
    elif 2020 <= x:
        return 2020

In [86]:
df.개봉일.apply(years).value_counts()

2010    11347
2000     2818
2020     2559
1990     1095
1900      818
1980      507
Name: 개봉일, dtype: int64